In [ ]:
from pyspark.sql import types
from pyspark.sql.functions import col
import ts.flint
from ts.flint import FlintContext
from ts.flint import summarizers

In [ ]:
fc = FlintContext(sqlContext)

In [ ]:
def date_parser(fmt):
    @ts.flint.udf(types.LongType())
    def parse(x):
        dt = datetime.datetime.strptime(str(x), fmt)
        return int(dt.strftime("%s%f")) * 1000
    return parse

In [ ]:
weather = (sqlContext.read.csv('weather.csv', header=True, inferSchema=True)
           .withColumn('time', date_parser('%Y%m%d')(col('DATE'))))
spy = (sqlContext.read.csv('spy.csv', header=True, inferSchema=True)
       .withColumn('time', date_parser('%Y-%m-%d %H:%M:%S')(col('DATE'))))

In [ ]:
weather_df = fc.read.dataframe(weather, is_sorted=False)
spy_df = fc.read.dataframe(spy, is_sorted=False)

In [ ]:
joined = spy_df.leftJoin(weather_df, tolerance="3d")
joined = joined.withColumn('change', joined.Close - joined.Open)

In [ ]:
joined.summarize(summarizers.linear_regression('change', ['PRCP', 'SNOW'])).toPandas()